# Mean centering and standard deviation normalization
In this chapter we are applying Mean centering and standard deviation normalization to the image data, so the model can better learn the data.

In [1]:
import pandas as pd
from os import chdir, listdir
import zipfile
import os
import cv2

import matplotlib.pyplot as plt
import numpy as np
import random

import tensorflow as tf
from tensorflow.keras import layers, models, preprocessing
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.losses import MeanSquaredError

# Import zip with the data
The data is imported as a zip from the github of our project group. The zip is unpacked in the google colab, so the data is accesible. 

In [ ]:
# Code from: https://towardsdatascience.com/an-informative-colab-guide-to-load-image-datasets-from-github-kaggle-and-local-machine-75cae89ffa1e

# Get zip file from Github URL
!wget --no-check-certificate \
    "https://github.com/ilsefeenstra/Fitgirls0011/archive/refs/heads/main.zip" \
    -O "/tmp/pawpularity_data.zip"

# Opens the zip file in read mode and extract files into /tmp folder
zip_ref = zipfile.ZipFile('/tmp/pawpularity_data.zip', 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

'wget' is not recognized as an internal or external command,
operable program or batch file.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\CASSMA~1\AppData\Local\Temp/ipykernel_2252/918870898.py", line 7, in <module>
    zip_ref = zipfile.ZipFile('/tmp/pawpularity_data.zip', 'r')
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\zipfile.py", line 1240, in __init__
    self.fp = io.open(file, filemode)
FileNotFoundError: [Errno 2] No such file or directory: '/tmp/pawpularity_data.zip'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception o

# Import tabular data

The tabular data is imported. This contains information on whether several elements are present in the image, such as blur, a human, a group, etc. Also the pawpularity score of the training data is in the table. For the test data only the image ID and the features are in the table. There is also a sample submission table, which contains the pawpularity score for the test data.

In [ ]:
# Import the CSV tables
csv_train_data = pd.read_csv("/tmp/Fitgirls0011-main/petfinder-pawpularity-score/train.csv")
csv_test_data = pd.read_csv("/tmp/Fitgirls0011-main/petfinder-pawpularity-score/test.csv")
sample_submission = pd.read_csv("/tmp/Fitgirls0011-main/petfinder-pawpularity-score/sample_submission.csv")
csv_train_data.head()

# Drop rows with missing values (if NaN values are in dataframe)
# No missing values present, so no samples dropped
csv_train_data.dropna()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



ERROR! Session/line number was not unique in database. History logging moved to new session 339
Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\CASSMA~1\AppData\Local\Temp/ipykernel_2252/202465788.py", line 2, in <module>
    csv_train_data = pd.read_csv("/tmp/Fitgirls0011-main/petfinder-pawpularity-score/train.csv")
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\pandas\util\_decorators.py", line 311, in wrapper
    return func(*args, **kwargs)
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\pandas\io\parsers\readers.py", line 586, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\pandas\io\parsers\readers.py", line 482, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "

In [ ]:
# create a plot that shows the distribution of the output of the training samples
plt.hist(csv_train_data['Pawpularity'], bins=100)
plt.title("Data distribution of the tabular data")
plt.xlabel("Pawpularity score")
plt.ylabel("Occurence")
plt.xlim(0, 100)

plt.show()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\CASSMA~1\AppData\Local\Temp/ipykernel_2252/496057712.py", line 2, in <module>
    plt.hist(csv_train_data['Pawpularity'], bins=100)
NameError: name 'csv_train_data' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed

# Import image data
The images are imported from the folders. Each image is reshaped to a 64x64 image. In this way all the images have the same shape and we do not use much memory, to speed up analysis. After the images are imported, the images and their names are shuffled. This is done, so we can later take a validation sample containing a random subsample of the dataset. It could be that the images in the dataset contain some order, so by shuffling we ensure that the subset for the validation data is random.


In [ ]:
def reshape_images(path, n):
    """
    This function returns a list of images, which are reshaped to 64 x 64 
    and a list with the names of the images.
    """
    # Set the current path
    chdir(path)
    
    # Preset the lists
    images = []
    image_names = []
    
    # Go over all the files in the path
    for i in listdir():
        
        # Get the name of the image, without .jpg
        image_names.append(i[:-4])
        
        # Get the image and reshape to n x n
        file = cv2.imread(i)
        file = cv2.resize(file,(n, n), interpolation=cv2.INTER_AREA)
        
        # Rescale the pixels and store in the list
        images.append(file/255)
        
    return images, image_names

# Reshape train and test images
train_imgs, train_names = reshape_images("/tmp/Fitgirls0011-main/petfinder-pawpularity-score/train", 64)
test_imgs, test_names = reshape_images("/tmp/Fitgirls0011-main/petfinder-pawpularity-score/test", 64)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\CASSMA~1\AppData\Local\Temp/ipykernel_2252/3063576225.py", line 29, in <module>
    train_imgs, train_names = reshape_images("/tmp/Fitgirls0011-main/petfinder-pawpularity-score/train", 64)
  File "C:\Users\CASSMA~1\AppData\Local\Temp/ipykernel_2252/3063576225.py", line 7, in reshape_images
    chdir(path)
FileNotFoundError: [WinError 3] Het systeem kan het opgegeven pad niet vinden: '/tmp/Fitgirls0011-main/petfinder-pawpularity-score/train'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'FileNotFoundError' object has no 

In [ ]:
# shuffle train image data
c = list(zip(train_imgs, train_names))

# make sure shuffle is the same each time
random.seed(4)
random.shuffle(c)

train_imgs, train_names = zip(*c)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\CASSMA~1\AppData\Local\Temp/ipykernel_2252/1203567697.py", line 2, in <module>
    c = list(zip(train_imgs, train_names))
NameError: name 'train_imgs' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerframe

# Combine tabular data with images
To ensure that the dataframe has the same order as the images in the list, we sort the dataframe based on the names of the images. If this would not be the case, it could be that you learn incorrectly, as the output of an image perhaps is not the real output.

In [ ]:
def sort_dataframe(data, images, names):
    """
    This function sorts the dataframe of the csv data according to the image names.
    """
    data_sorted = pd.DataFrame()

    # Iterate over images and get index of each image
    for img, name in zip(images, names):
        location = data[data['Id'] == name].index[0]

        # Sort dataframe according to index of images
        data_sorted = data_sorted.append([data.loc[location]])
        data_sorted = data_sorted.reset_index().drop(['index'],axis=1)
        
    return data_sorted

# Sort training and testing data
train_data_sorted = sort_dataframe(csv_train_data, train_imgs, train_names)
test_data_sorted = sort_dataframe(csv_test_data, test_imgs, test_names)
sample_submission_sorted = sort_dataframe(sample_submission, test_imgs, test_names)

train_data_sorted.head()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\CASSMA~1\AppData\Local\Temp/ipykernel_2252/222492375.py", line 18, in <module>
    train_data_sorted = sort_dataframe(csv_train_data, train_imgs, train_names)
NameError: name 'csv_train_data' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\ultratb.py", line 1101, in ge

# Processing data
First we split the data in a train and validation set, where 20% of the data is put in the validation set. Then the image data is converted to numpy arrays, so they can be used in the neural networks. The tabular data is split in the x and y values and also converted to numpy arrays.

In [ ]:
# split the dataset into training and validation set
split_threshold = int(0.2 * len(train_names))

val_tabular = train_data_sorted[:split_threshold]
train_tabular = train_data_sorted[split_threshold:]

val_images = train_imgs[:split_threshold]
train_images = train_imgs[split_threshold:]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\CASSMA~1\AppData\Local\Temp/ipykernel_2252/28358071.py", line 2, in <module>
    split_threshold = int(0.2 * len(train_names))
NameError: name 'train_names' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinne

In [ ]:
# Make numpy arrays of images for further analysis
train_imgs_array = np.array(train_images)
val_imgs_array = np.array(val_images)
test_imgs_array = np.array(test_imgs)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\CASSMA~1\AppData\Local\Temp/ipykernel_2252/757937732.py", line 2, in <module>
    train_imgs_array = np.array(train_images)
NameError: name 'train_images' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return _fixed_getinnerf

In [ ]:
# Select x-values (the 12 input features) from training data
x_train_tabular = train_tabular.iloc[:,1:13].to_numpy()

# Select y-values (pawpularity) from training data
y_train = train_tabular.iloc[:,13].to_numpy()

# select x and y values of validation dataset
x_val_tabular = val_tabular.iloc[:,1:13].to_numpy()
y_val = val_tabular.iloc[:,13].to_numpy()

# Select x-values (12 input features) from testing data
x_test_tabular = test_data_sorted.iloc[:,1:13].to_numpy()

# Select y-values (pawpularity) from testing data
y_test = sample_submission_sorted.iloc[:,1].to_numpy()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\CASSMA~1\AppData\Local\Temp/ipykernel_2252/2435366433.py", line 2, in <module>
    x_train_tabular = train_tabular.iloc[:,1:13].to_numpy()
NameError: name 'train_tabular' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\ultratb.py", line 1101, in get_records
    return 

# Create seperate neural networks
We create a tabular neural network to handle the data in the csv. Then we create a convolutional neural network to handle the image data. Both neural networks have no output layer, since they will be concatenated to one neural network, which will give the output.

In [2]:
def build_neural_net(input_size, hidden_nodes):
    """
    Build neural network with an input size and a hidden layer with a number of 
    hidden nodes.
    """
    # Create a sequential model object
    model = models.Sequential()

    # Create hidden layer 1 with relu activations
    model.add(layers.Dense(units=hidden_nodes, activation='relu', input_shape=(input_size,)))    

    return model

# Create neural network for tabular data and get summary of model 
# with 12 inputs and 100 hidden nodes
tabular_NN = build_neural_net(12, hidden_nodes=20)
tabular_NN.summary()

ERROR! Session/line number was not unique in database. History logging moved to new session 347
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                260       
Total params: 260
Trainable params: 260
Non-trainable params: 0
_________________________________________________________________


In [3]:
def build_convol_net(image_size, hidden_nodes):
    """
    Build neural network with an input size and a hidden layer with a number 
    of hidden nodes.
    """
    # Create a sequential model object
    model = models.Sequential()
    
    # Create a convolutional layer 1 with relu activation
    model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu', input_shape=image_size, padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2,2), strides=2))

    # Create a convolutional layer 2 with relu activation
    model.add(layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu', padding='same'))
    model.add(layers.MaxPool2D(pool_size=(2,2), strides=2))
    
    # Create a flattening layer
    model.add(layers.Flatten())

    # Create a dense layer with relu activations
    model.add(layers.Dense(units=hidden_nodes, activation="relu"))
    
    return model

In [4]:
# Create neural network for image data and get summary of model
image_size = (64, 64, 3)
image_NN = build_convol_net(image_size, hidden_nodes=20)
image_NN.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 64)        1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 32, 32, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 128)       0         
_________________________________________________________________
flatten (Flatten)            (None, 32768)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 20)                655380    
Total params: 731,028
Trainable params: 731,028
Non-trainable params: 0
________________________________________________

## Concatenate tabular and image data models
Concatenate the tabular and image models to create one neural network that can handle both types of data. This neural network will give the prediction of the pawpularity.

In [5]:
def concatenate_models(model1, model2, hidden_nodes):
    """
    Concatenate two neural network models, model1 and model2, and create
    a concatenated model with dense layers with some hidden nodes.
    """
    # Input for concatenated model is retrieved by concatenating the output
    # of both models
    concat_input = layers.concatenate([model1.output, model2.output])

    # Create hidden layers of the concatenated model
    hidden_layer_1 = layers.Dense(hidden_nodes, activation="relu")(concat_input)
    hidden_layer_2 = layers.Dense(hidden_nodes, activation="relu")(hidden_layer_1)
    output_layer = layers.Dense(1, activation="linear")(hidden_layer_2)

    # Create concatenated model with inputs of both models and output of the
    # concatenated model
    concat_model = models.Model(inputs=[model1.input, model2.input], outputs=output_layer)

    return concat_model

In [ ]:
# Part of code from: https://www.tensorflow.org/tutorials/keras/regression

def plot_loss(history):
    """
    Plot loss during epochs of training a neural network.
    """
    
    fig, axs = plt.subplots(1,2,figsize=(20,5)) 

    for i, metric in enumerate(['loss', 'root_mean_squared_error']):
        axs[i].plot(history.history[metric])
        axs[i].plot(history.history['val_'+metric])
        axs[i].legend(['training', 'validation'], loc='best')

        axs[i].set_title('Model '+metric)
        axs[i].set_ylabel(metric)
        axs[i].set_xlabel('epoch')

    plt.show()


def train_and_evaluate(model, image_x, tabular_x, train_y, x_val_tabular, x_val_imgs, val_y, epochs=20, preprocess = {}, augment={}):
  # Compile model and use mean squared error as loss and root mean squared error as metric
  model.compile(loss=MeanSquaredError(), metrics=[RootMeanSquaredError()])

  # preprocess
  train_gen = preprocessing.image.ImageDataGenerator(**preprocess, **augment)
  train_gen.fit(image_x)

  val_gen = preprocessing.image.ImageDataGenerator(**preprocess)
  val_gen.fit(image_x)

  # Train the model by fitting both tabular and image data at the same time
  history = model.fit(train_gen.flow([image_x, tabular_x], train_y), epochs=epochs, validation_data=val_gen.flow([x_val_imgs, x_val_tabular], val_y))

  # plot the loss and metric
  plot_loss(history)
  print(f"Validation Accuracy: {model.evaluate(val_gen.flow([x_val_imgs, x_val_tabular], val_y))[1]}")

# Concatenate tabular and image neural networks
concat_model = concatenate_models(image_NN, tabular_NN, hidden_nodes=20)

# Train model on both tabular and image data
train_and_evaluate(concat_model, train_imgs_array, x_train_tabular, y_train, 
                   x_val_tabular, val_imgs_array, y_val,
                   preprocess={'featurewise_center': True, 'featurewise_std_normalization': True}, epochs = 40)

# # See how the model scored
# loss = concat_model.evaluate([test_imgs_array, x_test_tabular], y_test)

# # Print to 3 decimals
# print(f'Test loss: {loss}')


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 3441, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\CASSMA~1\AppData\Local\Temp/ipykernel_2252/1561605700.py", line 44, in <module>
    train_and_evaluate(concat_model, train_imgs_array, x_train_tabular, y_train,
NameError: name 'train_imgs_array' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\interactiveshell.py", line 2061, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Cass Maes\anaconda3\envs\progLab\lib\site-packages\IPython\core\ultratb.py", line 1101, i

In [6]:
# Create overview of layers in model
tf.keras.utils.plot_model(concat_model, to_file='model.png', show_shapes=True, show_layer_names=True, rankdir='TB')

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
ERROR! Session/line number was not unique in database. History logging moved to new session 348
